In [191]:
# import necessary libraries
import pandas as pd
import os
import textstat
from openai import OpenAI
import json
import re
import requests
from dotenv import load_dotenv
import math

In [165]:
# import prompts 
from jh_pfx_prompts import example, icd10_example, baseline_zeroshot_prompt, single_fewshot_icd10_labeling_prompt

In [166]:
os.environ['OPENAI_API_KEY'] = ''

In [167]:
# api key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLIENT = OpenAI(api_key = OPENAI_API_KEY)
OPENAI_MODEL = "gpt-4o"

In [168]:
#reading levels
PROFESSIONAL = "Professional"
COLLEGE_GRADUATE = "College Graduate"
COLLEGE = "College"
TENTH_TO_TWELTH_GRADE = "10th to 12th grade"
EIGTH_TO_NINTH_GRADE = "8th to 9th grade"
SEVENTH_GRADE = "7th grade"
SIXTH_GRADE = "6th grade"
FIFTH_GRADE = "5th grade"
N_A = "N/A"

In [169]:
# https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch_reading_ease
def map_reading_level(flesch_reading_ease):
    if flesch_reading_ease < 10:
        return PROFESSIONAL
    elif 10.0 <= flesch_reading_ease < 30.0:
        return COLLEGE_GRADUATE
    elif 30.0 <= flesch_reading_ease < 50.0:
        return COLLEGE
    elif 50.0 <= flesch_reading_ease < 60.0:
        return TENTH_TO_TWELTH_GRADE
    elif 60.0 <= flesch_reading_ease < 70.0:
        return EIGTH_TO_NINTH_GRADE
    elif 70.0 <= flesch_reading_ease < 80.0:
        return SEVENTH_GRADE
    elif 80.0 <= flesch_reading_ease < 90.0:
        return SIXTH_GRADE
    elif 90.0 <= flesch_reading_ease < 100.0:
        return FIFTH_GRADE 
    else:
        return N_A

In [170]:
# reading ease variables
fifth_grade = 95
sixth_grade = 85
seventh_grade = 75
eigth_and_ninth_grade = 65
tenth_to_twelfth_grade = 55
college = 40
college_graduate = 20
professional = 5

In [171]:
def adjust_difference(diff, threshold):
    """Adjust the readability difference based on the threshold."""
    if diff > threshold:
        return diff - threshold
    return 0

In [172]:
# import fewshot examples
df_fewshot = pd.read_csv('pfx_fewshot_examples_college.csv')

In [173]:
# import evaluation data 
df_eval = pd.read_csv('pfx_evaluation_data.csv')

In [175]:
# extract the json from openai
def extract_json(openai_response):
    if openai_response:  # Ensure the response is not None
        try:
            # Extract content from response object
            content = openai_response.message.content
            
            # Search for JSON within the content
            json_match = re.search(r'```json\n(.*?)\n```', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                return json.loads(json_str)  # Parse JSON string to Python dict
            else:
                print("No JSON found in response content.")
                return None
        except AttributeError as e:
            print(f"Attribute error: {e}. Ensure the input is a valid response object.")
            return None
    else:
        return None

In [176]:
# takes json object and generated icd10 labels for incidental findings 
def label_icd10s(pfx_outputs_json):
    pfx_icd10_fewshot_examples = ""
    for i, row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)
    pfx_icd10_codes = []
    pfx_icd10_response = None 
    for pfx_output in pfx_outputs_json :
        try:
            prompt = single_fewshot_icd10_labeling_prompt.format(examples = pfx_icd10_fewshot_examples, PFx = pfx_output['PFx'])
        except:
            print("ERROR: %S" % pfx_output)
            return
        pfx_icd10_response = CLIENT.chat.completions.create(
            model = OPENAI_MODEL,
            temperature = 0.0,
            messages = [
                {"role": "system", "content": "You are an ICD10 medical coder for incidental findings. "},
                {"role": "system", "content": prompt}
            ],
            stream = False,
        )
    pfx_icd10_codes.append(pfx_icd10_response.choices[0])
    return list(map(extract_json, pfx_icd10_codes))

In [89]:
def label_icd10s(pfx_output):
    """
    Takes a single PFx response (string or JSON) and returns
    a labeled ICD-10 result as a Python dictionary (or object).
    """

    # Build up the few-shot examples for ICD-10 labeling
    pfx_icd10_fewshot_examples = ""
    for i, row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)

    # Generate the prompt for ICD-10 labeling
    # (Adjust the '{PFx}' if pfx_output is a dictionary with a specific key you need)
    prompt = single_fewshot_icd10_labeling_prompt.format(
        examples=pfx_icd10_fewshot_examples,
        PFx=pfx_output  # or PFx=pfx_output['key'] if needed
    )

    # Call the model to get ICD-10 codes
    pfx_icd10_response = CLIENT.chat.completions.create(
        model=OPENAI_MODEL,
        temperature=0.0,
        messages=[
            {
                "role": "system",
                "content": "You are an ICD10 medical coder for incidental findings."
            },
            {
                "role": "system",
                "content": prompt
            }
        ],
        stream=False,
    )

    # Extract the JSON structure (or dictionary) from the LLM response
    labeled_result = extract_json(pfx_icd10_response.choices[0])

    return labeled_result


In [213]:
# gets zeroshot response 
pfx_zeroshot_outputs = []
for i, row in df_eval.iloc[24:25].iterrows():
        prompt = baseline_zeroshot_prompt.format(Incidental_Finding = row['Incidental_Finding'], Reading_Level = COLLEGE)
pfx_response = None 
pfx_response = CLIENT.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = 0.0,
    messages = [
        {"role": "system", "content": "You are a md doctor rephrasing and explaining medical terminology to a patient in an understandable manor."},
        {"role": "system", "content": prompt}
    ],
    stream = False,
)

pfx_zeroshot_outputs.append(pfx_response.choices[0])

In [214]:
# converts to a readable format 
pfx_zeroshot_outputs_json = list(map(extract_json, pfx_zeroshot_outputs))

In [215]:
pfx_zeroshot_outputs_json

[{'finding': 'Bladder diverticulum',
  'ICD10_code': 'N32.3',
  'PFx': "A bladder diverticulum is a condition where a small pouch forms in the bladder wall. This pouch can collect urine, which might not empty completely when you urinate. This can sometimes lead to infections or other issues. It's like having a little pocket in your bladder that shouldn't be there. While it might not cause any symptoms, it's important to be aware of it because it can sometimes lead to complications. If you experience any unusual symptoms like frequent urination or discomfort, it's a good idea to let your healthcare provider know.",
  'PFx_ICD10_code': 'R39.15'}]

In [216]:
pfx_zeroshot_output_df = pd.DataFrame(pfx_zeroshot_outputs_json)

In [217]:
pfx_zeroshot_output_df

,finding,ICD10_code,PFx,PFx_ICD10_code
0,Bladder diverticulum,N32.3,A bladder diverticulum is a condition where a ...,R39.15


In [218]:
pfx_zeroshot_outputs_icd10_labels = label_icd10s(pfx_zeroshot_outputs_json)

In [219]:
pfx_zeroshot_output_df['_0_agent_icd10_codes'] = list(map(lambda x: list(x.values())[0] if x else "", pfx_zeroshot_outputs_icd10_labels))
pfx_zeroshot_output_df["_0_icd10_matches"]= pfx_zeroshot_output_df.ICD10_code == pfx_zeroshot_output_df._0_agent_icd10_codes
pfx_zeroshot_output_df["_0_pfx_icd10_matches"] = pfx_zeroshot_output_df.ICD10_code == pfx_zeroshot_output_df["PFx_ICD10_code"] 
pfx_zeroshot_output_df["_0_flesch"] = pfx_zeroshot_output_df['PFx'].apply(textstat.flesch_reading_ease)
pfx_zeroshot_output_df["_0_reading_level"] = pfx_zeroshot_output_df['_0_flesch'].apply(map_reading_level)
pfx_zeroshot_output_df["_0_flesch_kincaid"] = pfx_zeroshot_output_df['PFx'].apply(textstat.flesch_kincaid_grade)
pfx_zeroshot_output_df["_0_reading_level_matches"] = pfx_zeroshot_output_df['_0_reading_level'] == TENTH_TO_TWELTH_GRADE

In [237]:
desired_reading_ease = sixth_grade
# Calculate threshold for penalty
    if desired_reading_ease >= 55:
        threshold = 10
    else:
        threshold = 20

# Calculate accuracy score
accuracy_icd10_matches = sum(pfx_zeroshot_output_df["_0_icd10_matches"]) / len(pfx_zeroshot_output_df.index)
accuracy_pfx_matches = sum(pfx_zeroshot_output_df["_0_pfx_icd10_matches"]) / len(pfx_zeroshot_output_df.index)
flesch_score = pfx_zeroshot_output_df["_0_flesch"]

# Adjust weights for overall score
# total number of icd10 matches
total_icd10_matches = accuracy_icd10_matches + accuracy_pfx_matches

# Calculate readability score 
readability_score = flesch_score
readability_difference = abs(readability_score - desired_reading_ease)

# Compute the overall score
overall_score = total_icd10_matches * 0.8  + 0.2 * (1/(readability_difference + 1))

# Calculate readability score
readability_difference_log = desired_reading_ease - flesch_score
if readability_difference_log <= threshold:  # No penalty if difference is within the threshold
    readability_difference_p = 0
else:  # Apply penalty only if readability exceeds the threshold
    readability_difference_with_threshold = readability_difference_log - threshold
    readability_difference_p = math.log(1 + readability_difference_with_threshold) / math.log(20)

log_overall_score = total_icd10_matches * 0.8 + readability_difference_log * 0.2

grades_data = []
grades_data.append({
    "accuracy_agent_icd10": float(accuracy_icd10_matches),  # Assuming accuracy is already a scalar
    "accuracy_pfx_icd10": float(accuracy_pfx_matches),
    "readability_difference": float(readability_difference),  # Use .iloc[0] to extract scalar
    "overall_score": float(overall_score),  # Use .iloc[0] to extract scalar
    "log_overall_score": log_overall_score_list,
}) 
grades = pd.DataFrame(grades_data)
pfx_zeroshot_output_df = pd.concat([pfx_zeroshot_output_df, grades], axis = 1)


20.590000000000003
20.590000000000003


In [ ]:
pfx_zeroshot_output_df 

In [240]:
grades

,accuracy_agent_icd10,accuracy_pfx_icd10,readability_difference,overall_score
0,1.0,0.0,20.59,23.314947
